# CSCI 4181/6802: Tutorial 1
## K Nearest Neighbors for Protein Classification

Version: 1.2 (January 2018)

This is the first of four tutorials in the course that will give you practical experience with biological data. The exercises can be completed during class or later this week. The due date for this assignment is **Thursday, February 15, 2018**, at midnight (i.e., at the end of next Thursday). Please email your completed tutorial assignment to me at **mike.hall@dal.ca**. You have a few options for this. You can write up your responses in a standard word processor document (.doc/.docx/.txt), but the preferred method is to edit this document directly and send it either as a .ipynb or .html file (click File > Download As...). Just ensure your complete answers are included in whichever document you choose to send. Questions that you need to answer are indicated in **boldface**.

Each question is worth a single point. You can discuss the assignment with your classmates, but the work done and all submitted answers must be your own. Everyone has a unique data set, so don't be surprised if your results are different! Keep in mind that many of these questions have more than one “right” answer, and in some cases might have different answers depending on the reasoning you use. This is fine as long as you use good justifications for your answers.

The goal of this tutorial is to familiarize you with an important source for biological sequence data, particularly if you are looking for complete genome sequences or the complete set of proteins that are produced by an organism (the *proteome*). You will also use a simple k-nearest-neighbors (KNN) implementation to try and distinguish among two classes of sequences.

Each of you will be using KNN to try and classify the proteins from two different microorganisms. As I said in class, although everyone has a genome made of the same stuff (DNA nucleotides; A, C, G, and T), different organisms will have different proportions of these nucleotides in their genomes, and they will have them arranged in different ways. Since genes in DNA ultimately encode proteins, **these differences in nucleotide usage can influence the amino acid composition of an organism's proteins**. So the question we will ask today is, "how well can we tell the proteins of organisms A and B apart, based on their amino acid composition?" Lots of fun approaches have been developed to do this (including our own RITA, http://kiwi.cs.dal.ca/Software/index.php/RITA, http://nar.oxfordjournals.org/content/40/14/e111.long), but we’re going to keep it simple today.

### Software Pre-requisites

Python 3.6 is required for this tutorial.

If you are using Windows or Mac, I recommend anaconda: https://www.anaconda.com/download/. It is easy to install and gives you all of the bits you will need for these interactive tutorials.

If you are using Linux, I recommend using anaconda/miniconda to set up an environment for Python 3.6, especially if your system interpreter is Python 2.

We require the Python library `scikit-learn` (http://scikit-learn.org/stable/), as it contains the k-nearest-neighbours implementation that we will use. If using conda/miniconda, this can be installed with `conda install scikit-learn`, or otherwise with pip using the command `pip install scikit-learn`. This requires numpy, which should be installed automatically.

We will manage some of the data using the `pandas` package. It is included with anaconda, but can be installed with `conda install pandas` or `pip install pandas`.

In these tutorials, whenever you see a code block, like the one below, you must enter it by clicking on it and execute it by hitting Shift+Enter. This will change the [ ] icon to the left to [\*], indicating that the code is running. It will indicate a number once the code has finished executing. **Be sure to run every code block in order, as they depend on one another.**

The next code block loads all of the required modules. If you have any installation issues, the following code block will generate an error. Make sure to run this first and e-mail mike.hall@dal.ca if you are having any issues.

In [1]:
# If you can run this code block without errors, you have everything you need!
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

### Data Pre-requisites

We will be fetching protein data from the National Center for Biotechnology Information (NCBI). This is one of the main repositories for all types of DNA and protein sequence data.

The website where data can be retrieved is http://www.ncbi.nlm.nih.gov. I encourage you to browse the site after the tutorial, particularly the ‘nucleotide’ and ‘protein’ databases. This site also has a Web service for the BLAST database searching algorithm, which will be very useful to us in the future. Our entry point today, though, is going to be through their portal of genomes: 

http://www.ncbi.nlm.nih.gov/genome/browse/ 

This will give you an enormous list of genome projects in various stages of completion. Since we’re going to focus on prokaryotes today, click on the “prokaryotes” tab, which will bring up a list of all prokaryotic genome projects. Again, these are in various stages of completion, from “No data” to “Complete”, as indicated by the "Level" column with pie charts. After selecting the "Filter" button, there are checkboxes at the top that allow you to restrict the list to genomes in different stages of progress. You can also subselect based on phylum (“Group”) or class (“SubGroup”) and view the details of each organism’s genome by clicking on its name.

Clicking on the link for an individual organism can sometimes give you useful information. For example, if you click on the link for _Aeropyrum pernix_, you get a genome summary page that includes a short description of the organism’s lifestyle and links to papers that may contain more useful information.

At this stage, it would take too long to select reasonable sets of proteins directly from NCBI, so I have built sets for each of you beforehand. If you are interested, the data preparation code is in Tutorial1_Appendix.ipynb. Here's a description of the data preparation steps: First, we download a reference *E. coli* genome from NCBI's FTP servers, along with several other interesting microbial genomes. Next, we create BLAST (an alignment tool) databases for each genome. We filter the *E. coli* reference set to include only proteins involved in translation by searching for the terms 'tRNA', 'translation', 'ribosome', or 'ribosomal' in the sequence label. Using the BLAST local alignment tool (more on this later in the course), we search each of the *E. coli* proteins against the proteins from the other genomes. For each *E. coli* reference protein, we retain the gene that is the closest match in the other genomes. These are the subsets that we will be working with, stored in standard FASTA format.

## Tutorial Questions

Each of you have been assigned a distinct pair of genomes. You can find out which two genomes you have been assigned by putting your Banner ID in the code below and hitting Shift+Enter. The results will change from my genome pair to yours. **Be sure to change the Banner ID below to your Banner ID. This will automatically select your unique genome pair in all following code blocks.**

In [2]:
%run Tutorial1_Appendix.ipynb #Fetch some functions from the Appendix, such as get_genome_pair(student_id)
my_genomes = get_genome_pair("B00672276")
print(my_genomes) # Print the genome IDs out as text below

('GCA_001553935', 'GCA_000011805')


These are identifiers known as "accessions". They uniquely identify a record in the genome database. They aren't very useful for determining which microorganism these genomes belong to, however. Let's dig into the reference FASTA files that have been provided and look at the first line for each of the two genomes:

In [3]:
with open("tutorial1_appendix_data/" + my_genomes[0] + "_subset.faa", 'r') as genome_file:
    print(genome_file.readline())
    
with open("tutorial1_appendix_data/" + my_genomes[1] + "_subset.faa", 'r') as genome_file:
    print(genome_file.readline())

>AMD98026.1 50S ribosomal protein L27 [Actinomyces oris]

>AAW84497.1 tRNA modification GTPase TrmE [Vibrio fischeri ES114]



These are FASTA file records that describe the sequence. The first bit is the protein sequence accession, followed by a description of the protein, and then the microorganism genus and species (and sometimes strain) in square brackets. My genomes are from _Actinomyces oris_ and _Vibrio fischeri_.

**Q1. Identify the names of the two organisms from the first lines of your files. Collect and report the following information about each of your two organisms:**

- ** Three pieces of information about its lifestyle and/or genome (temperature preference, whether it causes disease, how big its genome is, whether it has plasmids, etc.). This information can be retrieved from resources such as MicrobeWiki and journal articles which you should be able to find in PubMed and Google Scholar. In any case, please tell me where you found this information. **

- ** One journal article about each organism. You can find this by searching the PubMed database at NCBI or Google Scholar for articles containing your organism name in the title or abstract. Cite these papers in MLA format (tip: http://scholar.google.com, search the article, and click the small quotation marks for automatically generated citations): **
  - Sambhara, Suryaprakash, and Gregory A. Poland. "H5N1 Avian influenza: preventive and therapeutic strategies against a pandemic." *Annual review of medicine* 61 (2010): 187-198.

_Response for question 1_

**Q2. I have selected a subset of proteins to use (translation-related proteins), rather than the complete proteome of each organism. **

**Why can it be advantageous to use a subset of proteins rather than the complete set? **

**Why might proteins involved in translation (particularly proteins in the ribosome) be a particularly good subset to use instead of another system such as degradation of a particular carbohydrate? (Hint: Our goal is to be able to compare very distantly related organisms, so consider why translation-related proteins might make sense for this goal).**

_Response for question 2_



Our goal is to see if we can distinguish between our microorganisms if we are given a translation-related gene of it. We need to select a feature of these genes to train and classify with, so we have chosen to count amino acid "words" of length `word_size`. There are two functions that we need here. `count_words` takes in a sequence, slides across it one character at a time and returns a dictionary that has counted the words of length `w`. `index_fasta_file` takes in a FASTA filename and a word size and runs `count_words` on each sequence in a FASTA file. It stores the counts for each protein in a pandas DataFrame object, which is a convenient table data structure for doing data analysis.

**This code is provided mainly for transparency. Look at it only if you're curious, but be sure to run it to load the functions!**

In [4]:
def count_words(sequence, w):
    count_dict = {}
    for i in range(0, len(sequence) - w + 1):
        word = sequence[i:i+w]
        if word not in count_dict:
            count_dict[word] = 1
        else:
            count_dict[word] += 1
    return count_dict

def index_fasta_file(fasta_filename, w = 4):
    word_dict = {} # Stores each of the protein data points
    sequence = "" # Start building the sequence, since it can be spread across multiple lines
    row = 0 # Keep track of which sequence we have processed
    with open(fasta_filename, 'r') as fasta_file:
        for line in fasta_file: # Loop through the file line-by-line
            if line[0] == ">": # Every time it hits a label, we are done processing the last sequence
                if len(sequence) >= w:
                    # Count the words using the function above
                    count_dict = count_words(sequence, w)
                    # For each word, we want to add it to our matrix that we are building
                    for word, count in count_dict.items():
                        # If we've seen the word before, append to the existing count list
                        if word in word_dict:
                            word_dict[word] = word_dict[word].append(
                                                              pd.Series([count],index=[row])
                                                                    )
                        # If it's a new word, start a counting list
                        else:
                            word_dict[word] = pd.Series([count],index=[row])
                    row += 1
                sequence = ""
            else: # We have a protein sequence, not a label
                sequence += line.strip() # Build the full sequence string, removing trailing whitespace with strip() 
    return pd.DataFrame(word_dict).fillna(0) # Return the counts as a DataFrame

Now that we have functions to handle our file input and word counting, we can load and process our protein data. In the following code block, you can change `word_size` to change the size of the amino acid "words" we are using as features for the KNN classifier. Beware, this is extremely inefficient code, so values >= 5 can produce more than 20^5 = 3.2 million indices, which takes a lot of time and RAM.

In [5]:
word_size = 1 # Word size: Change this to change the word length for each genome

# Provide a warning in case someone tries to run this for large values of word_size
if word_size >= 5:
    print("Warning: This code is written for clarity, not efficiency, so this could take a lot of time and RAM.")
    
count_matrix_1 = index_fasta_file("tutorial1_appendix_data/" + my_genomes[0] + "_subset.faa", word_size)
count_matrix_2 = index_fasta_file("tutorial1_appendix_data/" + my_genomes[1] + "_subset.faa", word_size)

# Create ground-truth labels for each protein, a list that looks like [0,0,0, ... ,0,1,1,1, ... ,1,1,1]
# 0 indicates genome 1, and 1 indicates genome 2
genome_labels = [0]*count_matrix_1.shape[0] + [1]*count_matrix_2.shape[0]

Now that our data are in the right format, we can train them with scikit-learn's classifier. The toolkit provides a useful `train_test_split` function that randomly selects a given proportion (set to 0.33 here) of the data to withhold as a test set. Parameters in the codeblock below that you can play with: 
- `test_size` defines the proportion of the data that should be withheld for the test set, with the remainder going into the training set
- `n_neighbors` defines the number of neighbors to check for a consensus for the KNN algorithm.

In [6]:
# Combine our two genome count tables into one table, making sure to fill NAs with 0
# NAs can occur when we append matrices with different sets of words (i.e., a word was observed in one genome)
# but not the other)
combined_matrix = count_matrix_1.append(count_matrix_2).fillna(0)

def KNN(X, y):
    # Normalize within samples (i.e., proteins) so that the length of the proteins doesn't impact the distance measure
    normalized_X = normalize(X, axis=1)

    # Split the data points randomly, witholding 25% of the data to test on
    X_train, X_test, y_train, y_test = train_test_split(normalized_X, y, test_size = 0.33)

    # Initialize a classifier for KNN, with n_neighbors set to 3 to start
    neigh = KNeighborsClassifier(n_neighbors = 3)
    # Fit the model with the training split
    neigh.fit(X_train, y_train)
    # Test the model with the test split
    train_score = neigh.score(X_train, y_train)
    # Test the model with the test split
    test_score = neigh.score(X_test, y_test)

    print("Training set score: %.3f" % (train_score,))
    print("Test set score: %.3f" % (test_score, ))

# Call the classifier on our matrix and genome labels
KNN(combined_matrix, genome_labels)

Training set score: 0.887
Test set score: 0.818


The score is the proportion of proteins that can have their organism identified by a consensus of the K nearest neighbours.

**Q3. By re-running the classifier a number of times, we can get different accuracy scores on different random test/training sets. This is a useful procedure, but it differs from cross-validation. In what way is it different?  Which do you think is better and why? **

_Response for question 3_

Try running the code with a word size of 1 and various numbers of neighbours (between 3 and 10, but feel free to experiment). You can experiment also with the `test_size` parameter, as there is no hard-and-fast rule for balancing training and test sets, but 2:1 is often used.

**Q4. Run your choice of n_neighbors with five random test/training sets, and report in a table the training and test set accuracy scores. Without using a formal statistical procedure, do you think the training scores are better than, equal to, or worse than the test scores? Explain why you think they are different or similar. Note the number of neighbours you used. **

_Response for question 4_

| Replicate |   Training    | Test  |
| :-------- | ------------- | ----- |
| 1         |               |       |
| 2         |               |       |
| 3         |               |       |
| 4         |               |       |
| 5         |               |       |

**Q5. The accuracy on the test set is the best indicator of generalization ability. But some of you might find that the test scores are higher than the training set scores. In other words, the classifier performs better on cases it *hasn’t* seen than on cases it *has* seen. How can this be? For this combination of data set and classifier, there are a few reasons why this might be the case.**

_Response for question 5_

Now go back up and re-run the code blocks after changing the word size to 2, 3, or 4, doing five replicates of each. At a word size of 1, we have up to 20 words (or features), but with a word size of 4 we have 20^4 = 160,000 features, so be patient (each run took about a minute on my laptop). If you have a very old laptop, just run this up to a word size of 3.

**Q6. As before, report your training and test set scores in a table. You may or may not see that the training set scores are lower than those obtained with the simpler file! This might seem strange, given that in class I told you that more variables = more opportunities for overfitting. Since the original variables are still in the mix, how is it possible that we might see a decrease in training set accuracy here? Hint: it has something to do with a fundamental difference between k-nearest neighbours and other methods like decision trees, ANNs and SVMs.**

_Response for question 6_

| Replicate | Word Size | Training Score | Test Score |
| :-------- | :-------- | -------------- | ---------- |
| 1         | 2         |                |            |
| 2         | 2         |                |            |
| 3         | 2         |                |            |
| 4         | 2         |                |            |
| 5         | 2         |                |            |
| 1         | 3         |                |            |
| 2         | 3         |                |            |
| 3         | 3         |                |            |
| 4         | 3         |                |            |
| 5         | 3         |                |            |
| 1         | 4         |                |            |
| 2         | 4         |                |            |
| 3         | 4         |                |            |
| 4         | 4         |                |            |
| 5         | 4         |                |            |

It’s probably not the case that every variable (frequency of A, etc.) is going to be useful for classification (though hopefully some will be!) A feature selection approach might be worthwhile. If we were brave, we might try to do this on with 160,000 variables, but for sanity’s sake we will examine only the smaller set of with the word size equal to 1. Although relevance and redundancy are both important, let’s focus on relevance. 

**Q7. Given the nature of the predictor variables (real values), and the class response (binary), what would be a reasonable, specific criterion to compare the relevance of the 20 different predictor variables?**

_Response for question 7_

We can dig into the values for each of the features (amino acids, in the case of word_size=1). Here is some code to show the frequencies of each letter in both genomes, sorted from highest to lowest.

In [7]:
print("Combined Frequencies:")
# The `apply` function is applying `sum` to the columns (features, i.e., words)
# We can reverse the sorting to pull out the worst features by changing ascending to True
sorted_features = combined_matrix.apply(sum, axis=0).sort_values(ascending=False)
print(sorted_features)

# Change the range to select a different number of features
subset_features = sorted_features[0:5].index
print("\nSelecting only features: %s\n" % (subset_features,))
# Re-run the KNN classification using only this subset of the matrix
KNN(combined_matrix[subset_features], genome_labels)

Combined Frequencies:
A    7536.0
L    6379.0
V    5772.0
G    5769.0
E    5670.0
R    5164.0
D    4529.0
K    4115.0
T    3916.0
I    3675.0
S    3616.0
P    3249.0
Q    2518.0
F    2329.0
N    2237.0
M    1811.0
Y    1720.0
H    1681.0
W     766.0
C     628.0
dtype: float64

Selecting only features: Index(['A', 'L', 'V', 'G', 'E'], dtype='object')

Training set score: 0.819
Test set score: 0.739


Apply your criterion and use it to rank the 20 variables in terms of their relevance. You can do this be changing the function `sum` to some function that produces the feature importance criterion you have selected. If you are having trouble coding this or finding an appropriate function (or if your idea is more complex than a single function), e-mail mike.hall@dal.ca and we can try to work through a solution.  Choose the best 5 variables (corresponding to five amino acid frequencies) according to your ranking criterion. Try another 5 replicates of the KNN function using only this subset. As a complement to this, try the same analysis using the _worst_ five variables.

**Q8. Show the rankings for your amino acid subsets, in table form as above. Compare these results against each other, and against the full word_size=1 results from question 4. Are the training and/or the test set results different from each other, and from the run with the full set of variables?**

_Response for question 8_

**Q9. In this tutorial we used protein sequences, but we could have also used the DNA sequences of the genes that encode these proteins. Would you expect the results with DNA sequence to be better or worse than the results with proteins? Why?**

_Response for question 9_

**Q10. Given the phylogenetic relationship between the two organisms you have been assigned, would you expect the classification to be easy or challenging? Why? Did your results reflect this hypothesis?**

_Response for question 10_

That’s it! The recovered accuracies may be uninspiring, which may reflect limitations of our encoding scheme, shortcomings of the k-nearest neighbour method, or maybe even a lack of distinguishing signals within your specific data. Hopefully you have gained a sense of how to improve on the first point using different feature creation, selection, and extraction methods, how to explore different levels of bias and variance, and how to detect when the third factor may be limiting your classification accuracy.